In [31]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

import glob as glob
import os as os
import subprocess as sp
import shutil
import json
from tqdm import tqdm

import logging
from importlib import reload
reload(logging)
logging.basicConfig(
    format='%(asctime)s %(message)s',
    datefmt='%Y-%m-%d %I:%M:%S %p',
    level=logging.DEBUG)
logging.info('Started logging...')

import parmed as pmd
import pytraj as pt

from setup.anchor_atoms import grep_anchor_atoms
from setup.restraints import setup_static_restraints, setup_conformation_restraints, setup_guest_restraints, setup_guest_wall_restraints
from paprika.analysis import fe_calc



2018-08-09 02:20:25 PM Started logging...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [10]:
systems = """./a-bam-p
./a-bam-s
./a-but-p
./a-but-s
./a-cbu-p
./a-chp-p
./a-cbu-s
./a-chp-s
./a-cpe-p
./a-coc-p
./a-coc-s
./a-cpe-s
./a-hep-p
./a-ham-s
./a-ham-p
./a-hep-s
./a-hp6-p
./a-hex-p
./a-hex-s
./a-hp6-s
./a-hx2-p
./a-hpa-s
./a-hpa-p
./a-hx2-s
./a-mba-p
./a-hx3-s
./a-hx3-p
./a-mba-s
./a-mhp-p
./a-mha-p
./a-mha-s
./a-mhp-s
./a-nmh-p
./a-nmb-p
./a-nmb-s
./a-nmh-s
./a-oct-p
./a-oam-p
./a-oam-s
./a-oct-s
./a-pnt-p
./a-pam-p
./a-pam-s
./a-pnt-s
./b-ben-s
./a-xxxx-s
./b-ben-p
./b-cbu-p
./b-cbu-s
./b-chp-s
./b-chp-p
./b-coc-s
./b-coc-p
./b-cpe-s
./b-cpe-p
./b-ham-s
./b-ham-p
./b-hep-s
./b-hep-p
./b-hex-p
./b-hex-s
./b-m4c-s
./b-m4c-p
./b-m4t-p
./b-m4t-s
./b-mch-s
./b-mha-s
./b-mha-p
./b-mch-p
./b-mo3-s
./b-mo4-p
./b-mo4-s
./b-mo3-p
./b-mp3-s
./b-mp4-s
./b-mp4-p
./b-mp3-p
./b-oam-s
./b-pb3-s
./b-pb3-p
./b-oam-p
./b-pb4-s
./b-pha-s
./b-pb4-p
./b-pha-p
./b-pnt-s
./b-pnt-p"""
systems = systems.split("\n")
systems = [i[2:] for i in systems]
systems = [i for i in systems if "xxxx" not in i]

In [13]:
attach_string = "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = "0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00"
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

release_fractions = attach_fractions[::-1]

windows = [len(attach_fractions), len(pull_distances), 0]
print(f"There are {windows} windows in this attach-pull-release calculation.")

There are [15, 46, 0] windows in this attach-pull-release calculation.


I only want to analyze the attach and pull phase for each of these. So it seems perhaps easiest to directly modify the release fractions.

In [99]:
for system in systems:

    if os.path.exists(f"{system}-results.json"):
        print(f"Skipping {system}")
        continue
    else:
        print(f"Running {system}")
    
    structure = pmd.load_file(
        os.path.join("systems", system, "smirnoff", "smirnoff.prmtop"),
        os.path.join("systems", system, "smirnoff", "smirnoff.inpcrd"),
        structure=True,
    )

    anchor_atoms = grep_anchor_atoms(system)
    static_restraints = setup_static_restraints(
        anchor_atoms, windows, structure, distance_fc=5.0, angle_fc=100.0
    )

    guest_restraints = setup_guest_restraints(
        anchor_atoms,
        windows,
        structure,
        attach_fractions,
        distance_fc=5.0,
        angle_fc=100.0,
        pull_initial=6.0,
        pull_final=24.0,
    )

    host_conformational_template = [["O5", "C1", "O1", "C4"], ["C1", "O1", "C4", "C5"]]
    host_conformational_targets = [104.30, -108.8]

    conformational_restraints = setup_conformation_restraints(
        host_conformational_template,
        host_conformational_targets,
        windows,
        attach_fractions,
        structure,
        resname="MGO",
        fc=6.0,
    )

    guest_wall_template = [
        ["O2", anchor_atoms["G1"]],
        ["O6", anchor_atoms["G1"]],
        [anchor_atoms["D2"], anchor_atoms["G1"], anchor_atoms["G2"]],
    ]
    guest_wall_targets = [11.3, 13.3, 80.0]

    guest_wall_restraints = setup_guest_wall_restraints(
        guest_wall_template,
        guest_wall_targets,
        structure,
        windows,
        resname="MGO",
        angle_fc=500.0,
        distance_fc=50.0,
    )

    restraints = (
        static_restraints
        + conformational_restraints
        + guest_restraints
        + guest_wall_restraints
    )

    for restraint in restraints:
        restraint.phase["release"]["force_constants"] = None
        restraint.phase["release"]["targets"] = None
    
    #
    #
    #
    structure = pt.load(
        os.path.join("systems", system, "smirnoff", "a000", "smirnoff.inpcrd"),
        os.path.join("systems", system, "smirnoff", "a000", "smirnoff.prmtop"),
    )

    analyze = fe_calc()
    analyze.prmtop = structure.topology
    analyze.trajectory = "prod.*.nc"
    analyze.path = os.path.join("systems", system, "smirnoff")

    analyze.restraint_list = guest_restraints + conformational_restraints
    analyze.collect_data()
    analyze.methods = ["ti-block"]
    analyze.quicker_ti_matrix = True
    analyze.bootcycles = 1000
    analyze.compute_free_energy(phases=["attach", "pull"])
    analyze.compute_ref_state_work([guest_restraints[0], guest_restraints[1], None, None, guest_restraints[2], None])
    with open(f"{system}-results.json", "w") as f:
        dumped = json.dumps(analyze.results, cls=NumpyEncoder)
        f.write(dumped)


Skipping a-bam-p
Skipping a-bam-s
Skipping a-but-p
Skipping a-but-s
Skipping a-cbu-p
Skipping a-chp-p
Skipping a-cbu-s
Skipping a-chp-s
Skipping a-cpe-p
Skipping a-coc-p
Skipping a-coc-s
Skipping a-cpe-s
Skipping a-hep-p
Skipping a-ham-s
Skipping a-ham-p
Skipping a-hep-s
Skipping a-hp6-p
Skipping a-hex-p
Skipping a-hex-s
Skipping a-hp6-s
Skipping a-hx2-p
Skipping a-hpa-s
Skipping a-hpa-p
Skipping a-hx2-s
Skipping a-mba-p
Skipping a-hx3-s
Skipping a-hx3-p
Skipping a-mba-s
Skipping a-mhp-p
Skipping a-mha-p
Skipping a-mha-s
Skipping a-mhp-s
Skipping a-nmh-p
Skipping a-nmb-p
Skipping a-nmb-s
Skipping a-nmh-s
Skipping a-oct-p
Skipping a-oam-p
Skipping a-oam-s
Skipping a-oct-s
Skipping a-pnt-p
Skipping a-pam-p
Skipping a-pam-s
Skipping a-pnt-s
Skipping b-ben-s
Skipping b-ben-p
Skipping b-cbu-p
Skipping b-cbu-s
Skipping b-chp-s
Skipping b-chp-p
Skipping b-coc-s
Skipping b-coc-p
Skipping b-cpe-s
Skipping b-cpe-p
Skipping b-ham-s
Skipping b-ham-p
Skipping b-hep-s
Skipping b-hep-p
Skipping b-hex

In [30]:
analyze.results["attach"]["ti-block"]["fe"]
analyze.results["pull"]["ti-block"]["fe"]

5.3557557010515895

In [82]:
import base64
import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):

    def default(self, obj):
        """If input object is an ndarray it will be converted into a dict 
        holding dtype, shape and the data, base64 encoded.
        """
        if isinstance(obj, np.ndarray):
            if obj.flags['C_CONTIGUOUS']:
                obj_data = obj.data
            else:
                cont_obj = np.ascontiguousarray(obj)
                assert(cont_obj.flags['C_CONTIGUOUS'])
                obj_data = cont_obj.data
            data_b64 = base64.b64encode(obj_data)
            # obj_data = obj.tolist()
            return dict(__ndarray__=data_b64.decode("utf-8"),
                        dtype=str(obj.dtype),
                        shape=obj.shape)
            # return dict(__ndarray__=obj_data,
            #             dtype=str(obj.dtype),
            #             shape=obj.shape)
        elif isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)):
            return obj.tolist()

        # Let the base class default method raise the TypeError
        # return json.JSONEncoder(self, obj) 
        return super(NumpyEncoder, self).default(obj)



def json_numpy_obj_hook(dct):
    """Decodes a previously encoded numpy ndarray with proper shape and dtype.

    :param dct: (dict) json encoded ndarray
    :return: (ndarray) if input was an encoded ndarray
    """
    if isinstance(dct, dict) and '__ndarray__' in dct:
        data = base64.b64decode(dct['__ndarray__'])
        return np.frombuffer(data, dct['dtype']).reshape(dct['shape'])
        # return dct['__ndarray__']
    return dct

expected = np.arange(100, dtype=np.float)
dumped = json.dumps(expected, cls=NumpyEncoder)
result = json.loads(dumped, object_hook=json_numpy_obj_hook)


# None of the following assertions will be broken.
assert result.dtype == expected.dtype, "Wrong Type"
assert result.shape == expected.shape, "Wrong Shape"
assert np.allclose(expected, result), "Wrong Values"


In [87]:
dumped = json.dumps(analyze.results, cls=NumpyEncoder)

In [89]:
with open("tmp.json", "w") as f:
    dumped = json.dumps(analyze.results, cls=NumpyEncoder)
    f.write(dumped)

In [79]:
loaded = json.loads(dumped, object_hook=json_numpy_obj_hook)


In [94]:
with open("tmp.json", "r") as f:
    data = f.read()
    loaded = json.loads(data, object_hook=json_numpy_obj_hook)

In [95]:
loaded

{'attach': {'window_order': array([ 0,  1,  2, ..., 12, 13, 14]),
  'ti-block': {'n_frames': 300000,
   'fraction_fe_matrix': {'1.0': array([[  0.        ,   0.70037679,   1.2104489 , ...,  10.01424067,
             11.75967157,  13.08656481],
           [ -0.70037679,   0.        ,   0.50419884, ...,   9.30799061,
             11.05342151,  12.38031475],
           [ -1.2104489 ,  -0.50419884,   0.        , ...,   8.79936976,
             10.54480065,  11.87169389],
           ..., 
           [-10.01424067,  -9.30799061,  -8.79936976, ...,   0.        ,
              1.73530067,   3.06219391],
           [-11.75967157, -11.05342151, -10.54480065, ...,  -1.73530067,
              0.        ,   1.31177856],
           [-13.08656481, -12.38031475, -11.87169389, ...,  -3.06219391,
             -1.31177856,   0.        ]])},
   'fraction_sem_matrix': {'1.0': array([[ 0.        ,  0.06994489,  0.0967811 , ...,  0.14380752,
             0.14533827,  0.1464889 ],
           [ 0.06994489,  0.

In [81]:
loaded

{'attach': {'window_order': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
  'ti-block': {'n_frames': 300000,
   'fraction_fe_matrix': {'1.0': [[0.0,
      0.7003767917927635,
      1.2104489018996711,
      1.9115592309288374,
      2.4052970957703064,
      3.1694212374781037,
      3.737866697940118,
      4.6513426659309305,
      5.35863245639219,
      6.493830187856018,
      7.421539087829302,
      8.891987678407567,
      10.01424067067177,
      11.759671565084682,
      13.086564807109855],
     [-0.7003767917927635,
      0.0,
      0.5041988439585569,
      1.2053091729877232,
      1.699047037829192,
      2.4631711795369893,
      3.031616639999004,
      3.9450926079898165,
      4.652382398451074,
      5.787580129914904,
      6.715289029888186,
      8.185737620466453,
      9.307990612730656,
      11.053421507143568,
      12.38031474916874],
     [-1.2104489018996711,
      -0.5041988439585569,
      0.0,
      0.6966883177673927,
      1.1904261826088613,
 

In [50]:
import base64
import json
import numpy as np

# This is bsaed off the `matplotlib.collections.pathcollection` code.
# https://github.com/mpld3/mpld3/issues/434#issuecomment-340255689

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    

In [40]:
class NumpyAwareJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray) and obj.ndim == 1:
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


In [44]:
j=json.dumps(analyze.results,cls=NumpyEncoder)


In [45]:
j

'{"attach": {"window_order": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], "ti-block": {"n_frames": 300000, "fraction_fe_matrix": {"1.0": [[0.0, 0.7003767917927635, 1.2104489018996711, 1.9115592309288374, 2.4052970957703064, 3.1694212374781037, 3.737866697940118, 4.6513426659309305, 5.35863245639219, 6.493830187856018, 7.421539087829302, 8.891987678407567, 10.01424067067177, 11.759671565084682, 13.086564807109855], [-0.7003767917927635, 0.0, 0.5041988439585569, 1.2053091729877232, 1.699047037829192, 2.4631711795369893, 3.031616639999004, 3.9450926079898165, 4.652382398451074, 5.787580129914904, 6.715289029888186, 8.185737620466453, 9.307990612730656, 11.053421507143568, 12.38031474916874], [-1.2104489018996711, -0.5041988439585569, 0.0, 0.6966883177673927, 1.1904261826088613, 1.9545503243166589, 2.5229957847786735, 3.4364717527694855, 4.143761543230744, 5.278959274694572, 6.206668174667857, 7.677116765246121, 8.799369757510325, 10.54480065192324, 11.87169389394841], [-1.911559230